In [1]:
### -*- coding: utf-8 -*-

# Created by: Raf

import os
import sys
import time
import threading
import pandas as pd

from PyQt5 import QtGui, QtWidgets, QtCore
from PyQt5.QtWidgets import QGraphicsView, QGraphicsScene, QGraphicsItem, QGraphicsPixmapItem, QInputDialog, QMessageBox
from PyQt5.QtGui import QPixmap, QIcon
from PyQt5.QtCore import QTime, QEventLoop
from MainWindowUI import Ui_Form

from douzero.env.game import GameEnv
from douzero.evaluation.deep_agent import DeepAgent

EnvCard2RealCard = {3: '3', 4: '4', 5: '5', 6: '6', 7: '7',
                    8: '8', 9: '9', 10: 'T', 11: 'J', 12: 'Q',
                    13: 'K', 14: 'A', 17: '2', 20: 'X', 30: 'D'}

RealCard2EnvCard = {'3': 3, '4': 4, '5': 5, '6': 6, '7': 7,
                    '8': 8, '9': 9, 'T': 10, 'J': 11, 'Q': 12,
                    'K': 13, 'A': 14, '2': 17, 'X': 20, 'D': 30}

AllEnvCard = [3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7,
              8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12,
              12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 17, 17, 17, 17, 20, 30]


class MyPyQT_Form(QtWidgets.QWidget, Ui_Form):
    def __init__(self):
        super(MyPyQT_Form, self).__init__()
        self.setupUi(self)
        self.setWindowFlags(QtCore.Qt.WindowMinimizeButtonHint |
                            QtCore.Qt.WindowCloseButtonHint |
                            QtCore.Qt.WindowStaysOnTopHint)
        self.setFixedSize(self.width(), self.height())
        self.setWindowIcon(QIcon('pics/favicon.ico'))
        window_pale = QtGui.QPalette()
        window_pale.setBrush(self.backgroundRole(), QtGui.QBrush(QtGui.QPixmap("pics/bg.png")))
        self.setPalette(window_pale)

        self.Players = [self.RPlayer, self.Player, self.LPlayer]
        self.counter = QTime()
        self.consecutive_passes = 0  # 防止连续过牌

        # 模型路径
        self.card_play_model_path_dict = {
            'landlord': "baselines/douzero_WP/landlord.ckpt",
            'landlord_up': "baselines/douzero_WP/landlord_up.ckpt",
            'landlord_down': "baselines/douzero_WP/landlord_down.ckpt"
        }

    # def init_display(self):
    #     self.WinRate.setText("胜率：--%")
    #     self.UserHandCards.setText("手牌")
    #     self.LPlayedCard.setText("上家出牌区域")
    #     self.RPlayedCard.setText("下家出牌区域")
    #     self.PredictedCard.setText("AI出牌区域")
    #     self.ThreeLandlordCards.setText("三张底牌")
    #     for player in self.Players:
    #         player.setStyleSheet('background-color: rgba(255, 0, 0, 0);')

    # def init_cards(self):
    #     # 创建手动输入对话框，请优化UI
    #     dialog = QtWidgets.QDialog(self)
    #     dialog.setWindowTitle("手动输入")
    #     dialog.setFixedSize(400, 300)

    #     layout = QtWidgets.QVBoxLayout()

    #     # 添加手牌输入框
    #     hand_label = QtWidgets.QLabel("请输入你的手牌(如: 3456789TJQKA2XD):")  # 手牌选择框，请建立UI界面
    #     self.hand_input = QtWidgets.QLineEdit()

    #     # 添加底牌输入框
    #     landlord_label = QtWidgets.QLabel("请输入三张底牌(如: 2XD):")  # 底牌选择框，请建立UI界面
    #     self.landlord_input = QtWidgets.QLineEdit()

    #     # 添加角色选择
    #     role_label = QtWidgets.QLabel("请选择你的角色:")  # 角色选择框，请优化UI
    #     self.role_combo = QtWidgets.QComboBox()
    #     self.role_combo.addItems(["地主上家", "地主", "地主下家"])

    #     # 确认按钮
    #     confirm_btn = QtWidgets.QPushButton("确认")
    #     confirm_btn.clicked.connect(lambda: self.on_input_confirm())

    #     layout.addWidget(hand_label)
    #     layout.addWidget(self.hand_input)
    #     layout.addWidget(landlord_label)
    #     layout.addWidget(self.landlord_input)
    #     layout.addWidget(role_label)
    #     layout.addWidget(self.role_combo)
    #     layout.addWidget(confirm_btn)

    #     dialog.setLayout(layout)
    #     dialog.exec_()

    def on_input_confirm(self):

        # 获取输入的手牌
        self.user_hand_cards_real = self.hand_input.text().upper()
        self.UserHandCards.setText(self.user_hand_cards_real)
        

        # 获取输入的底牌
        self.three_landlord_cards_real = self.landlord_input.text().upper()
        self.ThreeLandlordCards.setText("BCards：" + self.three_landlord_cards_real)


        # 获取选择的角色
        self.user_position_code = self.role_combo.currentIndex()
        self.user_position = ['landlord_up', 'landlord', 'landlord_down'][self.user_position_code]

        if not self.initialize_card_library():  # 初始化牌库失败则返回
            QMessageBox.critical(self, "Error", "Library Failed, Check input！")
            return

        self.user_hand_cards_env = [RealCard2EnvCard[c] for c in list(self.user_hand_cards_real)]
        self.three_landlord_cards_env = [RealCard2EnvCard[c] for c in list(self.three_landlord_cards_real)]
        
        # 隐藏初始输入及显示手牌位置
        self.hand_input.hide()
        self.landlord_input.hide()
        self.role_combo.hide()
        self.UserHandCards.show()
        
        # 更新界面显示
        for player in self.Players:
            player.setStyleSheet('background-color: rgba(255, 0, 0, 0);')
        self.Players[self.user_position_code].setStyleSheet('background-color: rgba(255, 0, 0, 0.1);')

        #dialog.close()
        self.process_after_input()

    def initialize_card_library(self):
        """初始化牌库，记录剩余牌数"""
        # 总牌库
        total_cards = ['3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A', '2', 'X', 'D']
        card_counts = {'3': 4, '4': 4, '5': 4, '6': 4, '7': 4, '8': 4, '9': 4, 'T': 4,
                      'J': 4, 'Q': 4, 'K': 4, 'A': 4, '2': 4, 'X': 1, 'D': 1}

        # 扣除玩家手牌
        for card in self.user_hand_cards_real:
            if card not in total_cards:
                QMessageBox.critical(self, "ERROR", f"Hand：Card {card} Not Existed！", QMessageBox.Yes)
                return False
            card_counts[card] -= 1
            if card_counts[card] < 0:
                QMessageBox.critical(self, "ERROR", f"Card {card} Not Enough！", QMessageBox.Yes)
                return False

        # 只有当玩家是地主时才扣除底牌
        if self.user_position == "landlord":
            for card in self.three_landlord_cards_real:
                if card not in total_cards:
                    QMessageBox.critical(self, "ERROR", f"Landlord {card} Not Exist！", QMessageBox.Yes)
                    return False
                card_counts[card] -= 1
                if card_counts[card] < 0:
                    QMessageBox.critical(self, "ERROR", f"Card {card} Not Enough！", QMessageBox.Yes)
                    return False

        self.card_library = card_counts
        self.update_card_library_display()
        return True

    def update_card_library_display(self):
        """更新牌库显示"""
        import pandas as pd
        cards = sorted(self.card_library.keys())
        counts = [self.card_library[card] for card in cards]
        df = pd.DataFrame([cards, counts], index=['牌', '剩余数量'])
        print("\n当前牌库剩余:")
        print(df)  
        remaining_text = ""    # .join([f"{card}: {count}" for card, count in zip(cards, counts)])
        for index, (card,count) in enumerate(zip(cards, counts)):
            remaining_text += f"{card}({count})  "
            if (index +1) % 5 == 0:
                remaining_text += "\n"
                
        self.RemainingCardsText.setText(remaining_text)  # 更新剩余牌数显示

    def validate_and_update_card_library(self, played_cards):
        """验证并更新牌库"""
        if not played_cards:  # 不出牌
            return True

        temp_lib = self.card_library.copy()
        for card in played_cards:
            if card not in temp_lib:
                QMessageBox.warning(self, "Error", f"Card {card} Not Existed！", QMessageBox.Yes)
                return False
            if temp_lib[card] <= 0:
                QMessageBox.warning(self, "Error", f"Card {card} Out Of Numbers！", QMessageBox.Yes)
                return False
            temp_lib[card] -= 1

        # 验证通过，更新牌库
        self.card_library = temp_lib
        self.update_card_library_display()
        return True

    def process_after_input(self):
        # 整副牌减去玩家手上的牌，就是其他人的手牌
        self.user_hand_cards_env = [RealCard2EnvCard[c] for c in list(self.user_hand_cards_real)]
        self.three_landlord_cards_env = [RealCard2EnvCard[c] for c in list(self.three_landlord_cards_real)]

        # 如果玩家是地主，将底牌加入手牌
        if self.user_position == "landlord":
            self.user_hand_cards_env.extend(self.three_landlord_cards_env)
            self.user_hand_cards_real += self.three_landlord_cards_real

        self.other_hand_cards = []
        for i in set(AllEnvCard):
            self.other_hand_cards.extend([i] * (AllEnvCard.count(i) - self.user_hand_cards_env.count(i)))

        # 修正手牌分配逻辑
        if self.user_position == "landlord":
            # 玩家是地主(20张)，其他两家各17张
            landlord_up_cards = self.other_hand_cards[:17]
            landlord_down_cards = self.other_hand_cards[17:]
        elif self.user_position == "landlord_up":
            # 玩家是地主上家(17张)，地主20张(17+3)，地主下家17张
            landlord_cards = self.other_hand_cards[:17] + self.three_landlord_cards_env
            landlord_down_cards = self.other_hand_cards[17:34]  # 取接下来的17张
        else:  # landlord_down
            # 玩家是地主下家(17张)，地主20张(17+3)，地主上家17张
            landlord_cards = self.other_hand_cards[:17] + self.three_landlord_cards_env
            landlord_up_cards = self.other_hand_cards[17:34]  # 取接下来的17张

        # 构建手牌数据字典
        self.card_play_data_list = {
            'three_landlord_cards': self.three_landlord_cards_env,
            'landlord_up': landlord_up_cards if self.user_position != "landlord_up" else self.user_hand_cards_env,
            'landlord': landlord_cards if self.user_position != "landlord" else self.user_hand_cards_env,
            'landlord_down': landlord_down_cards if self.user_position != "landlord_down" else self.user_hand_cards_env
        }

        # 校验手牌数量
        if len(self.card_play_data_list["three_landlord_cards"]) != 3:
            QMessageBox.critical(self, "Error", "Bottom Card Must In 3 Numbers！", QMessageBox.Yes, QMessageBox.Yes)
            self.init_display()
            return

        # 修改手牌数量校验逻辑
        expected_landlord_up = 17 if self.user_position != "landlord_up" else len(self.user_hand_cards_env)
        expected_landlord_down = 17 if self.user_position != "landlord_down" else len(self.user_hand_cards_env)
        expected_landlord = 20 if self.user_position != "landlord" else len(self.user_hand_cards_env)

        if len(self.card_play_data_list["landlord_up"]) != expected_landlord_up or \
                len(self.card_play_data_list["landlord_down"]) != expected_landlord_down or \
                len(self.card_play_data_list["landlord"]) != expected_landlord:
            QMessageBox.critical(
                self, 
                "Error", 
                f"Card Numbers Error (Upper:{len(self.card_play_data_list['landlord_up'])}, " +
                f"Lower:{len(self.card_play_data_list['landlord_down'])}, " +
                f"Landlord:{len(self.card_play_data_list['landlord'])})",
                QMessageBox.Yes
            )
            self.init_display()
            return

        # 得到出牌顺序
        self.play_order = 0 if self.user_position == "landlord" else 1 if self.user_position == "landlord_up" else 2

        # 创建一个代表玩家的AI
        ai_players = [0, 0]
        ai_players[0] = self.user_position
        ai_players[1] = DeepAgent(self.user_position, self.card_play_model_path_dict[self.user_position])

        self.env = GameEnv(ai_players)
        self.start()
    def start(self):
            self.env.card_play_init(self.card_play_data_list)
            print("Start Game\n")
            self.consecutive_passes = 0  # 初始化计数器
            self.first_round = True  # 初始化首回合标志
            while not self.env.game_over:
                if self.play_order == 0:
                    # 玩家出牌 - 自动获取AI建议
                    self.PredictedCard.setText("...")
                    action_message = self.env.step(self.user_position)
                    action_str = action_message["action"] if action_message["action"] else "Pass"

                    self.UserHandCards.setText("Cards：" + str(''.join(
                        [EnvCard2RealCard[c] for c in self.env.info_sets[self.user_position].player_hand_cards]))[::-1])
                    self.PredictedCard.setText(action_message["action"] if action_message["action"] else "Pass")
                    self.WinRate.setText("Rate：" + action_message["win_rate"])

                    QMessageBox.information(self, "Perdiction",
                                                f"AI Perdiction: {action_message['action'] if action_message['action'] else 'Pass'}\nRate: {action_message['win_rate']}",
                                                QMessageBox.Ok)
                    if action_str == "Pass":
                        self.consecutive_passes += 1
                    else:
                        self.consecutive_passes = 0
                    self.first_round = False
                    self.play_order = 1
                    self.HistoryList.addItem(f"Player Input: {action_str}")  # 添加历史记录

                elif self.play_order == 1:
                    # 下家出牌 - 手动输入
                    self.RPlayedCard.setText("...")
                    must_play = self.first_round
                    while True:
                        text, ok = QInputDialog.getText(self, "Lower", "Input Cards(345 or null means pass):")
                        if not ok:
                            return
                        if not text:  # 不出
                            if must_play and self.consecutive_passes == 0:
                                QMessageBox.warning(self, "Error", "1st Role Need Cards")  # 强制出牌
                                continue
                            else:
                                self.consecutive_passes += 1
                                self.other_played_cards_real = ""
                                break
                        # 验证牌型
                        if self.validate_card_pattern(text.upper()) and \
                                self.validate_and_update_card_library(text.upper()):
                            self.other_played_cards_real = text.upper()
                            self.consecutive_passes = 0
                            if self.first_round:
                                self.first_round = False
                            break
                        else:
                            QMessageBox.warning(self, "Error", "Not Card-types or Cards", QMessageBox.Yes)

                    self.other_played_cards_env = [RealCard2EnvCard[c] for c in list(self.other_played_cards_real)]
                    self.env.step(self.user_position, self.other_played_cards_env)
                    self.RPlayedCard.setText(self.other_played_cards_real if self.other_played_cards_real else "Pass")
                    self.play_order = 2
                    self.HistoryList.addItem(
                        f"Lower: {self.other_played_cards_real if self.other_played_cards_real else 'Pass'}")

                elif self.play_order == 2:
                    # 上家出牌 - 手动输入
                    self.LPlayedCard.setText("...")
                    must_play = self.first_round
                    while True:
                        text, ok = QInputDialog.getText(self, "Upper", "Input Cards(etc:345 or null means pass):")
                        if not ok:
                            return
                        if not text:  # 不出
                            if must_play and self.consecutive_passes == 0:
                                QMessageBox.warning(self, "Error", "1st Role Need Cards！")
                                continue
                            else:
                                self.consecutive_passes += 1
                                self.other_played_cards_real = ""
                                break
                        # 验证牌型
                        if self.validate_card_pattern(text.upper()) and \
                                self.validate_and_update_card_library(text.upper()):
                            self.other_played_cards_real = text.upper()
                            self.consecutive_passes = 0
                            if self.first_round:
                                self.first_round = False
                            break
                        else:
                            QMessageBox.warning(self, "Error", "Not Card-types or Cards", QMessageBox.Yes)

                    self.other_played_cards_env = [RealCard2EnvCard[c] for c in list(self.other_played_cards_real)]
                    self.env.step(self.user_position, self.other_played_cards_env)
                    self.LPlayedCard.setText(self.other_played_cards_real if self.other_played_cards_real else "Pass")
                    self.play_order = 0
                    self.HistoryList.addItem(
                        f"Upper: {self.other_played_cards_real if self.other_played_cards_real else 'Pass'}")

                self.counter.restart()
                while self.counter.elapsed() < 100:
                    QtWidgets.QApplication.processEvents(QEventLoop.AllEvents, 50)

            print("{} Win，End Game!\n".format("Farmer" if self.env.winner == "farmer" else "Landlord"))
            QMessageBox.information(self, "End Game", "{} Win！".format("Farmer" if self.env.winner == "farmer" else "Landlord"),
                                    QMessageBox.Yes, QMessageBox.Yes)
            self.env.reset()
            self.init_display()

    def validate_card_pattern(self, cards_str):
        """验证牌型是否符合规则"""
        if not cards_str:
            return True

        cards = list(cards_str)
        card_count = {}
        for card in cards:
            card_count[card] = card_count.get(card, 0) + 1
     
        #王炸    
        if len(cards) == 2 and set(cards) == {'X', 'D'}:
            return True
        
        # 单牌
        if len(cards) == 1:
            return True

        # 对子
        if len(cards) == 2 and len(card_count) == 1:
            return True

        # 三张
        if len(cards) == 3 and len(card_count) == 1:
            return True

        # 三带一
        if len(cards) == 4 and (list(card_count.values()).count(3) == 1 and list(card_count.values()).count(1) == 1):
            return True
        
        # 三带二
        if len(cards) == 5 and (list(card_count.values()).count(3) == 1 and list(card_count.values()).count(2) == 1):
            return True
        
        # 炸弹(四张相同)
        if len(cards) == 4 and len(card_count) == 1:
            return True

        # 顺子(至少5张连续单牌)
        if len(cards) >= 5 and len(card_count) == len(cards) and self.is_sequence(cards):
            return True

        # 连对(至少3对连续对子)
        if len(cards) >= 6 and len(cards) % 2 == 0 and all(
                count == 2 for count in card_count.values()) and self.is_sequence(sorted(card_count.keys())):
            return True
            
        if len(cards) >= 6 and len(cards) % 3 == 0:
            triplets = [card for card, count in card_count.items() if count == 3]
            if len(triplets) >= 2 and self.is_sequence(triplets):
                # 检查是否只有三张牌且没有其他牌型
                if len(triplets) * 3 == len(cards):
                    return True
                    
        # 四带二(4张相同+2张单牌)
        if len(cards) == 6 and (list(card_count.values()).count(4) == 1 and list(card_count.values()).count(1) == 2):
            return True

        # 飞机带单牌(连续的三带一)
        if len(cards) >= 8 and len(cards) % 4 == 0:
            triplets = [card for card, count in card_count.items() if count == 3]
            if len(triplets) >= 2 and self.is_sequence(triplets):
                singles = [card for card, count in card_count.items() if count == 1]
                if len(singles) == len(triplets):
                    return True

        # 飞机带对子(连续的三带二)
        if len(cards) >= 10 and len(cards) % 5 == 0:
            triplets = [card for card, count in card_count.items() if count == 3]
            if len(triplets) >= 2 and self.is_sequence(triplets):
                pairs = [card for card, count in card_count.items() if count == 2]
                if len(pairs) == len(triplets):
                    return True

        return False

    def is_sequence(self, cards):
        """检查是否是连续的牌"""
        order = ['3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A', '2']
        card_indices = [order.index(c) for c in cards]
        return all(card_indices[i] + 1 == card_indices[i + 1] for i in range(len(card_indices) - 1))

    def stop(self):
        try:
            self.env.game_over = True
        except AttributeError:
            pass


if __name__ == '__main__':
    os.environ["GIT_PYTHON_REFRESH"] = 'quiet'
    app = QtWidgets.QApplication(sys.argv)
    my_pyqt_form = MyPyQT_Form()
    my_pyqt_form.show()
    sys.exit(app.exec_())


当前牌库剩余:
     0  1  2  3  4  5  6  7  8  9  10 11 12 13 14
牌     2  3  4  5  6  7  8  9  A  D  J  K  Q  T  X
剩余数量  4  0  0  0  0  3  4  4  4  1  4  4  4  4  1
Start Game


当前牌库剩余:
     0  1  2  3  4  5  6  7  8  9  10 11 12 13 14
牌     2  3  4  5  6  7  8  9  A  D  J  K  Q  T  X
剩余数量  4  0  0  0  0  3  0  3  4  1  4  4  4  3  1


SystemExit: 0

D:\anaconda\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
